<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bea8b243cda7017e73d3c38415b48f31de4eebf1bf9e19524680cc218884a489
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-24 12:58:41
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17


-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: -12.43 K (-0.09%)
Current PnL: -19.64 L (-13.08%)
CY Booked + Current PnL: -7.54 L (-5.02%)
-------------------
Total profit:  2.24 L
Total loss:  -21.88 L
-------------------
Total Booked + Current PnL: 19.44 L (15.79%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.52%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.82 L (61.13%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-9.16,68.0,X-LC,3.20,227292.0,16926.0,11865.0,0.59,8.05,5.22,13.68,37.0,1.43,1.61,25.90,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,3.12,60.0,X-MC,2.80,216420.0,24678.0,13072.0,0.81,12.87,6.04,19.69,99.0,1.89,1.53,20.54,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-9.44,42.0,X-MC,3.33,89520.0,-1158.0,16069.0,-0.91,-1.28,17.95,16.44,101.0,-0.07,0.63,18.64,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,96.58,51.0,M-SC,1.96,84711.0,-16066.0,16163.0,0.75,-15.94,19.08,0.09,245.0,-0.99,0.60,11.29,OX40N,NTT,DURABLES
42,ITC,452.0,-37.08,62.0,X-LC,1.97,203106.0,2968.0,17914.0,-0.14,1.48,8.82,10.44,4.0,0.17,1.44,7.13,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-196.59,80.0,M-SC,24.53,167382.0,18162.0,126022.0,6.64,12.17,75.29,96.62,208.0,0.14,1.18,81.14,XY25,NTT,FINANCE
82,VAIBHAVGBL,521.00,56.47,62.0,H-SC,4.37,135614.0,-47161.0,158234.0,2.66,-25.80,116.68,60.77,125.0,-0.30,0.96,21.46,XR,NTT,JEWELLERY
70,SURYODAY,240.00,68.86,64.0,H-SC,12.25,155672.0,-23399.0,89122.0,2.56,-13.07,57.25,36.71,135.0,-0.26,1.10,54.46,XR,NTT,BANKS
36,IEX,219.00,-29.93,63.0,H-SC,17.40,200483.0,529.0,96693.0,2.23,0.26,48.23,48.63,115.0,0.01,1.42,11.85,XR,NTT,MISC
32,HINDZINC,730.22,31.07,55.0,M-LC,11.04,213346.0,8270.0,104305.0,1.72,4.03,48.89,54.89,52.0,0.08,1.51,29.10,X5K,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDUNILVR,2922.00,-10.50,43.0,X-LC,7.66,251180.0,-5883.0,41018.0,-3.45,-2.29,16.33,13.67,24.0,-0.14,1.78,17.65,XY25,NTT,FMCG
21,COLPAL,3767.14,-4.98,43.0,X-MC,8.18,221790.0,-41575.0,154920.0,-3.02,-15.79,69.85,43.04,84.0,-0.27,1.57,2.97,XY25,ATH,FMCG
7,ASIANTILES,137.00,7088.89,63.0,L-SC,7.20,78870.0,-14940.0,91560.0,-2.19,-15.93,116.09,81.67,269.0,-0.16,0.56,52.73,XR,NTT,CERAMICS
26,GREENPANEL,537.00,229.46,53.0,M-SC,5.35,147010.0,-36068.0,115050.0,-2.06,-19.70,78.26,43.14,230.0,-0.31,1.04,38.28,XY24,NTT,MISC
54,RAJESHEXPO,518.00,1655.05,45.0,L-SC,3.79,47893.0,-89284.0,89378.0,-1.62,-65.09,186.62,0.07,267.0,-1.00,0.34,19.11,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.0,41.94,48.0,M-MC,10.01,223860.0,-1102.0,168746.0,0.51,-0.49,75.38,74.53,192.0,-0.01,1.58,31.61,XY24,BTT,STEEL
36,IEX,219.0,-29.93,63.0,H-SC,17.40,200483.0,529.0,96693.0,2.23,0.26,48.23,48.63,115.0,0.01,1.42,11.85,XR,NTT,MISC
87,WIPRO,420.0,-13.79,48.0,M-LC,5.92,151516.0,571.0,108879.0,0.03,0.38,71.86,72.51,53.0,0.01,1.07,6.42,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,122.87,69.0,H-SC,15.27,140751.0,-1728.0,21253.0,0.44,-1.21,15.10,13.70,163.0,-0.08,0.99,66.12,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.87,59.0,H-SC,2.37,228150.0,-41517.0,77845.0,-0.76,-15.40,34.12,13.47,138.0,-0.53,1.61,16.01,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,96.58,51.0,M-SC,1.96,84711.0,-16066.0,16163.0,0.75,-15.94,19.08,0.09,245.0,-0.99,0.60,11.29,OX40N,NTT,DURABLES
73,TATAELXSI,9161.0,-22.03,56.0,H-MC,7.49,105488.0,-26347.0,68567.0,1.48,-19.98,65.00,32.03,98.0,-0.38,0.75,18.16,OX40N,NTT,IT
67,SIS,528.0,2072.02,52.0,H-SC,3.24,87656.0,-23376.0,46984.0,-0.35,-21.05,53.60,21.26,156.0,-0.50,0.62,18.43,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,48.0,M-LC,5.69,205672.0,2502.0,39633.0,-0.25,1.23,19.27,20.74,55.0,0.06,1.45,6.84,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.07,55.0,M-LC,11.04,213346.0,8270.0,104305.0,1.72,4.03,48.89,54.89,52.0,0.08,1.51,29.10,X5K,ATH,METALS
46,KPIGREEN,731.05,7.10,66.0,H-SC,5.18,127436.0,2139.0,56785.0,-0.57,1.71,44.56,47.03,141.0,0.04,0.90,59.56,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,60.0,M-LC,9.12,161795.0,20955.0,23541.0,-0.39,14.88,14.55,31.59,66.0,0.89,1.14,45.31,XY24,NTT,BANKS
87,WIPRO,420.00,-13.79,48.0,M-LC,5.92,151516.0,571.0,108879.0,0.03,0.38,71.86,72.51,53.0,0.01,1.07,6.42,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-20.50,69.0,H-MC,14.26,206046.0,26238.0,86168.0,-1.15,14.59,41.82,62.52,88.0,0.30,1.46,53.65,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.52,65.0,M-SC,3.83,103875.0,12376.0,66376.0,-1.03,13.53,63.90,86.07,223.0,0.19,0.73,50.85,XR,NTT,DURABLES
32,HINDZINC,730.22,31.07,55.0,M-LC,11.04,213346.0,8270.0,104305.0,1.72,4.03,48.89,54.89,52.0,0.08,1.51,29.10,X5K,ATH,METALS
46,KPIGREEN,731.05,7.10,66.0,H-SC,5.18,127436.0,2139.0,56785.0,-0.57,1.71,44.56,47.03,141.0,0.04,0.90,59.56,MH,ATH,POWER
37,INDIAMART,4810.62,-55.20,44.0,H-SC,7.16,125318.0,1982.0,129641.0,0.63,1.61,103.45,106.72,119.0,0.02,0.89,25.36,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
24,DMART,5391.45,-16.90,34.0,X-LC,10.17,101650.0,-936.0,27740.0,-0.76,-0.91,27.29,26.13,38.0,-0.03,0.72,24.43,X40N,ATH,FMCG
53,QUESS,986.00,-48.21,34.0,X-SC,38.76,51246.0,-13760.0,164689.0,-0.43,-21.17,321.37,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.18,41.0,X-MC,6.78,82250.0,-1260.0,21286.0,-0.81,-1.51,25.88,23.98,82.0,-0.06,0.58,4.22,X40,ATH,APPARELS
52,PGHH,17907.65,-29.62,42.0,X-MC,3.22,203850.0,3030.0,64763.0,-0.84,1.51,31.77,33.76,80.0,0.05,1.44,6.78,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.97,49.0,H-LC,0.79,157455.0,-28640.0,76114.0,-0.03,-15.39,48.34,25.51,15.0,-0.38,1.11,16.03,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,62.0,X-LC,1.97,203106.0,2968.0,17914.0,-0.14,1.48,8.82,10.44,4.0,0.17,1.44,7.13,X40,NTT,FMCG
85,VOLTAS,1530.00,3.12,60.0,X-MC,2.80,216420.0,24678.0,13072.0,0.81,12.87,6.04,19.69,99.0,1.89,1.53,20.54,XY25,NTT,AC
29,HAVELLS,2069.16,-5.33,50.0,X-MC,2.87,234811.0,-12920.0,87984.0,0.59,-5.22,37.47,30.30,92.0,-0.15,1.66,7.38,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-9.16,68.0,X-LC,3.20,227292.0,16926.0,11865.0,0.59,8.05,5.22,13.68,37.0,1.43,1.61,25.90,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,34.0,X-SC,38.76,51246.0,-13760.0,164689.0,-0.43,-21.17,321.37,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
21,COLPAL,3767.14,-4.98,43.0,X-MC,8.18,221790.0,-41575.0,154920.0,-3.02,-15.79,69.85,43.04,84.0,-0.27,1.57,2.97,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,48.0,X-MC,2.90,184480.0,-53471.0,206120.0,-0.59,-22.47,111.73,64.15,174.0,-0.26,1.30,3.48,XY24,BTT,CEMENT
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147305.0,-129218.0,239282.0,0.98,-46.73,162.44,39.81,54.0,-0.54,1.04,3.82,XY24,NTT,AUTO
51,PAGEIND,51769.93,-27.18,41.0,X-MC,6.78,82250.0,-1260.0,21286.0,-0.81,-1.51,25.88,23.98,82.0,-0.06,0.58,4.22,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,34.0,X-SC,38.76,51246.0,-13760.0,164689.0,-0.43,-21.17,321.37,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-27.18,41.0,X-MC,6.78,82250.0,-1260.0,21286.0,-0.81,-1.51,25.88,23.98,82.0,-0.06,0.58,4.22,X40,ATH,APPARELS
57,RELAXO,1176.00,-39.39,45.0,X-SC,4.68,82645.0,-62515.0,138447.0,-0.93,-43.07,167.52,52.31,136.0,-0.45,0.58,10.62,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.44,42.0,X-MC,3.33,89520.0,-1158.0,16069.0,-0.91,-1.28,17.95,16.44,101.0,-0.07,0.63,18.64,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.44,46.0,X-SC,4.44,96604.0,-32066.0,77360.0,-0.32,-24.92,80.08,35.20,219.0,-0.41,0.68,10.88,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-27.40,58.0,X-LC,12.33,289370.0,-56586.0,123301.0,0.17,-16.36,42.61,19.28,1.0,-0.46,2.05,6.98,X40,ATH,IT
40,INFY,2275.00,-16.73,59.0,X-LC,7.17,324825.0,11937.0,159749.0,-0.23,3.81,49.18,54.87,3.0,0.07,2.30,10.72,X40,BTT,IT
42,ITC,452.00,-37.08,62.0,X-LC,1.97,203106.0,2968.0,17914.0,-0.14,1.48,8.82,10.44,4.0,0.17,1.44,7.13,X40,NTT,FMCG
84,VBL,671.64,-18.08,53.0,X-LC,9.87,294788.0,-21054.0,133716.0,-0.75,-6.67,45.36,35.67,5.0,-0.16,2.08,6.26,X40N,ATH,FMCG
1,ABB,7934.00,-40.25,50.0,H-LC,6.31,249840.0,-11779.0,130991.0,0.45,-4.50,52.43,45.57,7.0,-0.09,1.77,6.31,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7088.89,63.0,L-SC,7.20,78870.0,-14940.0,91560.0,-2.19,-15.93,116.09,81.67,269.0,-0.16,0.56,52.73,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,26.96,80659.0,-32890.0,72940.0,-0.17,-28.97,90.43,35.27,268.0,-0.45,0.57,97.42,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,49.0,H-SC,9.56,91605.0,-6375.0,27976.0,-0.57,-6.51,30.54,22.05,152.0,-0.23,0.65,33.15,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-38.52,65.0,M-SC,3.83,103875.0,12376.0,66376.0,-1.03,13.53,63.90,86.07,223.0,0.19,0.73,50.85,XR,NTT,DURABLES
46,KPIGREEN,731.05,7.10,66.0,H-SC,5.18,127436.0,2139.0,56785.0,-0.57,1.71,44.56,47.03,141.0,0.04,0.90,59.56,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-196.59,80.0,M-SC,24.53,167382.0,18162.0,126022.0,6.64,12.17,75.29,96.62,208.0,0.14,1.18,81.14,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,122.87,69.0,H-SC,15.27,140751.0,-1728.0,21253.0,0.44,-1.21,15.10,13.70,163.0,-0.08,0.99,66.12,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.73,70.0,M-SC,9.60,83738.0,-17421.0,57419.0,0.09,-17.22,68.57,39.54,202.0,-0.30,0.59,18.08,AR,ATH,AUTO
5,ANGELONE,3033.00,16.33,70.0,X-SC,6.39,205525.0,14519.0,43181.0,-0.18,7.60,21.01,30.21,157.0,0.34,1.45,30.07,X40N,NTT,FINANCE
59,REPCOHOME,880.00,-55.29,72.0,H-SC,2.96,257088.0,-29417.0,286756.0,0.90,-10.27,111.54,89.82,134.0,-0.10,1.82,36.02,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.45
1,25,43.81
2,50,74.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.44
LC    32.75
MC    22.80
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.26
X40      16.02
XY25     12.27
XR       11.33
X40N      9.30
AR        7.92
OX40N     7.62
X200      1.77
X5K       1.51
SR        1.09
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.64
X-LC    21.90
X-MC    17.05
M-SC    12.72
M-LC     5.17
H-LC     4.65
X-SC     4.61
H-MC     3.87
M-MC     1.58
L-SC     1.47
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.66,-4.01,37.94
IT,12.76,-16.58,79.44
FINANCE,12.20,-9.48,59.11
MISC,6.95,-17.02,81.86
BANKS,6.61,-6.52,62.56
PAINTS,5.62,-16.02,33.31
ELECTRICAL,5.31,-9.54,47.97
INSURANCE,3.88,-0.90,38.52
AUTO,2.80,-44.79,105.22


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3290491.0,22
XR,1307203.0,14
AR,1176218.0,9
X40,778150.0,12
X40N,611603.0,8
OX40N,569457.0,10
XY25,447594.0,8
SR,266805.0,2
X5K,104305.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3374252.0,25
M-SC,1206260.0,15
X-MC,1122885.0,13
X-LC,1041796.0,13
X-SC,526144.0,6
H-MC,333190.0,3
H-LC,280347.0,3
M-LC,276358.0,4
L-SC,253878.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1167262.0      6
           AR         845438.0      5
           XR         773704.0      7
M-SC       XY24       694965.0      6
X-MC       XY24       585400.0      4
X-LC       X40        448063.0      5
           XY24       307433.0      2
X-SC       X40N       284095.0      4
H-SC       SR         266805.0      2
           OX40N      264258.0      4
X-MC       X40        252727.0      6
X-LC       X40N       210742.0      3
H-LC       AR         207105.0      2
H-MC       XY24       178455.0      1
M-MC       XY24       168746.0      1
X-MC       XY25       167992.0      2
X-SC       XY24       164689.0      1
L-SC       XR         164500.0      2
M-SC       OX40N      147254.0      4
           XY25       126022.0      1
           AR         123675.0      2
X-MC       X40N       116766.0      1
M-SC       XR         114344.0      2
M-LC       XR         108879.0      1
           X5K        104305.0      1
L-SC       OX40N       89378.0      1
H-MC       XR          86168.0      1
X-SC       X40         77360.0      1
X-LC       XY25        75558.0      3
H-LC       X200        73242.0      1
H-MC       OX40N       68567.0      1
L-MC       XR          59608.0      1
H-SC       MH          56785.0      1
M-LC       XY25        39633.0      1
L-LC       XY25        38389.0      1
M-LC       XY24        23541.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
